Tema2
Lidia Istrate
342C4


Functions for reading the input:

In [70]:
import re


#Representation
CONSTANT = "CONSTANT"
VARIABLE = "VARIABLE"
FUNCTION = "FUNCTION"
ATOM = "ATOM"
CONDITION = "CONDITION"
RULE = "RULE"
QUESTION = "QUESTION"
ANSWER = "ANSWER"
COMMENT = "COMMENT"
NOT = "NOT"
OR = "OR"
AND = "AND"




def make_const(value):
    return (CONSTANT, value)


def make_var(name):
    return (VARIABLE, name)


def make_function_call(function, args):
    return (FUNCTION, function, args)


def make_atom(predicate, args, trust_coef):
    return (ATOM, predicate, args, trust_coef)


def make_question(predicate, args):
    return (ATOM, predicate, args)


def make_condition(conditions):
    return (CONDITION, CONDITION, conditions)

def make_rule(atom, conditions, trust_coef):
    
    rule = []
    for c in conditions:
        rule.append(make_neg(c))
    rule.append(atom)
    
    return (OR, OR, rule, trust_coef)



def make_neg(sentence):
    return (NOT, NOT, sentence)

def make_or(sentences):
    return (OR, OR, sentences) 

def is_constant(f):
    return f[0] == CONSTANT


def is_variable(f):
    return f[0] == VARIABLE


def is_function_call(f):
    return f[0] == FUNCTION


def is_atom(f):
    return f[0] == ATOM


def is_question(f):
    return f[0] == QUESTION


def is_term(f):
    return is_constant(f) or is_variable(f) or is_function_call(f)

def is_sentence(f):
    return is_atom(f) or f[0] == CONDITION or f[0] == OR or f[0] == NOT


def has_args(f):
    return is_function_call(f) or is_sentence(f)


def get_value(f):
    if is_constant(f):
        return f[1]
    return None


def get_name(f):
    if is_variable(f):
        return f[1]
    return None


def get_head(f):
    if is_function_call(f) or is_atom(f):
        return f[1]
    if f[0] == CONDITION:
        return CONDITION
    if f[0] == OR:
        return OR
    if f[0] == NOT:
        return NOT
    return None


def get_args(f):
    if has_args(f):
        return f[2]
    return []

def get_coef(f):
    if is_atom(f) or is_rule(f):
        return f[3]
    return None

def replace_args(formula, new_args):
    return (formula[0], formula[1], new_args)

def get_type(line):
    
    if line[0] == "?":
        return QUESTION
    elif ":" in line and line[0] != ":":
        return RULE
    elif line[0] == ":":
        return ANSWER
    elif line[0] == "#":
        return COMMENT
    else:
        return ATOM
        

facts = []
rules = []
questions = []

def process_atom(line):
        
    predicate = ''
    current_string = ''
    args = []
    index = 0
    i = 0
    trust_coef = 1
    
    while i < len(line):
        
        if line[i] == '(' and predicate == '':
            predicate = current_string

            current_string = ''
            index = i+1
        
        elif line[i] == ',' or line[i] == ')':
            if current_string != '':
                if '?' in current_string:
                    var = make_var(current_string)
                    args.append(var)

                else:
                    const = make_const(current_string)
                    args.append(const)

                
            current_string = ''
        
        elif line[i] == ' ':
            index = i+1
        elif line[i] == '(' and predicate != '':
            j = index
            function_string = ''
            
            count = 0
            first_paranth = False
            ok = False
            
            while ok == False:
                if line[j] == ')':
                    count -= 1
                    if first_paranth == True and count == 0:
                        ok = True
                if line[j] == '(':
                    count += 1
                    first_paranth = True

                function_string += line[j]
                j = j + 1
            

            atom_f = process_atom(function_string)
            function = make_function_call(atom_f[1], atom_f[2])
            args.append(function)
            current_string = ''
            i = j-1

        elif line[i] == '%':
            i = i + 2
            coef = ''
            while i < len(line):
                coef = coef + line[i]
                i = i +1
            trust_coef = float(coef)
        else:
            current_string = current_string + line[i]
        i = i + 1
            

    return make_atom(predicate, args, trust_coef)
    
    
def process_rule(line):
    conditions = []
    i = 0
    
    trust_coef = 1
    
    current_string = ''
    while (line[i] + line[i+1] != ' :'):
        current_string = current_string + line[i]
        i = i + 1
    atom = process_atom(current_string)
    i = i + 3

    
    while i < len(line):
        current_string = ''
        count = 0
        first_paranth = False
        ok = False

        while ok == False:
            if line[i] == ')':
                count -= 1
                if first_paranth == True and count == 0:
                    ok = True
            if line[i] == '(':
                count += 1
                first_paranth = True

            current_string += line[i]
            i = i + 1    

        condition = process_atom(current_string)
        conditions.append(condition)

        if i < len(line) - 1:
            if line[i:len(line)].strip()[0] == '%':
                while line[i] != '%':
                    i = i + 1
                if line[i] == '%':
                    i = i + 2
                    coef = ''
                    while i < len(line):
                        coef = coef + line[i]
                        i = i + 1
                    trust_coef = float(coef)
                
        
        i = i + 2
    
    return make_rule(atom, conditions, trust_coef)

    
def read_input(filename):
    
    f = open(filename, "r")
    for line in f:
        if line and line != '\n':
            full_line = line
            line = line.split()
            

            line_type = get_type(line)
            
            if line_type == ATOM:
                atom = process_atom(full_line)
                facts.append(atom)
            elif line_type == QUESTION:
                q_line = full_line[2:len(full_line)]
                atom = process_atom(q_line)
                question = make_question(atom[1], atom[2])
                questions.append(question)
            elif line_type == RULE:
                rule = process_rule(full_line)
                rules.append(rule)            
                
    f.close()


def pFail(message, f):
    print(message + " <" + str(f) + ">")
    return False
def check_term(T):
    if is_constant(T):
        return ((get_value(T) is not None) or pFail("The value of the constant is None", T))
    if is_variable(T):
        return ((get_name(T) is not None) or pFail("The name of the variable is None", T))
    if is_function_call(T):
        return (not [t for t in get_args(T) if not check_term(t)] and       
    (get_head(T) is not None or pFail("Function is not callable", T)))
    return pFail("Term is not one of constant, variable or function call", T)
def check_atom(A):
    if is_atom(A):
        return (not [t for t in get_args(A) if not check_term(t)] and     
    (get_head(A) is not None or pFail("Predicate name is None", A)))
    return pFail("Is not an atom", A)
def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [CONDITION, OR]:
            return ((len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S))    
        and not [s for s in get_args(S) if not check_sentence(s)])
        if get_head(S) == NOT:
            #return ((len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) 
            return True
        #and check_sentence(get_args(S)[0]))
    return pFail("Not sentence or unknown type", S)

def add_statement(kb, statement):
    
    if check_sentence(statement):
        kb.append(statement)
        #print("OK: Added statement " + print_formula(statement, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(statement, True)+"><" + str(statement) + ">")
    return False
    
    
            
def is_positive_literal(L):
    return is_atom(L)
def is_negative_literal(L):
    return get_head(L) == NOT #and is_positive_literal(get_args(L)[0])
def is_literal(L):
    return is_positive_literal(L) or is_negative_literal(L)            




def get_premises(formula):
    premises = []
    if is_rule(formula):
        args = get_args(formula) 
        for i in args:
            if get_head(i) == NOT:
                premises.append(get_args(i))
    return premises

def get_conclusion(formula):
    if is_rule(formula):
        args = get_args(formula)
        return args[len(args) - 1]
    return None

def is_fact(formula):
    return is_positive_literal(formula)

def is_rule(formula):
    args = get_args(formula)
    for i in args:
        if is_negative_literal(i):
            return True
    return False


def equal_terms(t1, t2):
    if is_constant(t1) and is_constant(t2):
        return get_value(t1) == get_value(t2)
    if is_variable(t1) and is_variable(t2):
            return get_name(t1) == get_name(t2)
    if is_function_call(t1) and is_function_call(t2):
        if get_head(t1) != get_head(t2):
            return all([equal_terms(get_args(t1)[i], get_args(t2)[i]) for i in range(len(get_args(t1)))])
    return False

def is_equal_to(a1, a2):
    if not (is_atom(a1) and is_atom(a2) and get_head(a1) == get_head(a2) and len(get_args(a1)) == len(get_args(a2))):
        return False
    return all([equal_terms(get_args(a1)[i], get_args(a2)[i]) for i in range(len(get_args(a1)))])


def substitute(f, substitution):
    if substitution is None:
        return None
    if is_variable(f) and (get_name(f) in substitution):
        return substitute(substitution[get_name(f)], substitution)
    if has_args(f):
        return replace_args(f, [substitute(arg, substitution) for arg in get_args(f)])
    return f

def occur_check(v, t, subst):
    if v == t:
        return True
    if is_variable(t) and (get_name(t) in subst):
        return occur_check(v, substitute(t, subst), subst)
    if has_args(t):
        return any(occur_check(v, arg, subst) for arg in get_args(t))
        
    return False


def unify(f1, f2, subst = None):
    if subst is None:
        subst = {}
    
    stack = [(f1, f2)]
    
    while len(stack) > 0:
        (s, t) = stack.pop()
        
        while is_variable(s) and (get_name(s) in subst):
            s = substitute(s, subst)
        while is_variable(t) and (get_name(t) in subst):
            t = substitute(t, subst)
        if s != t:
            if is_variable(s):
                if occur_check(s, t, subst):
                    return False
                else:
                    subst[get_name(s)] = t
            elif is_variable(t):
                if occur_check(t, s, subst):
                    return False
                else:
                    subst[get_name(t)] = s
            elif has_args(s) and has_args(t) and len(get_args(s)) == len(get_args(t)):
                if get_head(s) == get_head(t):
                    for i in range(0, len(get_args(s))):
                        stack.append((get_args(s)[i], get_args(t)[i]))
                else:
                    return False
            else:
                return False
                 
    return subst


from copy import deepcopy

def apply_rule(rule, facts):
    premises = get_premises(rule)
    conclusion = get_conclusion(rule)
    ok_substs = [{}]
    facts_from_rule = []
    
    for premise in premises:
        intermediate_substs = []
        
        for ok_subst in ok_substs:
            for fact in facts:
                unify_result = unify(premise, fact, deepcopy(ok_subst))
                
                if unify_result != False:
                    intermediate_substs.append(unify_result)       
        ok_substs = intermediate_substs
        
    
    for substitution in ok_substs:
        facts_from_rule.append(substitute(conclusion, substitution))
    
    return facts_from_rule



def get_kb(rules, facts):
    kb = []
    
    for fact in facts:
        add_statement(kb, fact)

    for rule in rules:
        add_statement(kb, rule)
        
    return kb

def print_formula(f, return_result = False):
    ret = ""
    if is_term(f):
        if is_constant(f):
            ret += str(get_value(f))
        elif is_variable(f):
            ret += get_name(f)
        elif is_function_call(f):
            ret += str(get_head(f)) + "[" + "".join([print_formula(arg, True) + "," for arg in get_args(f)])[:-1] + "]"
        else:
            ret += "???"
    elif is_atom(f):
        ret += str(get_head(f)) + "(" + "".join([print_formula(arg, True) + ", " for arg in get_args(f)])[:-2] + ")"
    elif is_rule(f):
        # negation, conjunction or disjunction
        args = get_args(f)
        if len(args) == 1:
            ret += print_formula(get_conclusion(f))
        else:
            ret += print_formula(get_conclusion(f)) + " : "
            premises = get_premises(f)
            for premise in premises:
                ret += print_formula(premise, True)
                ret += ", "
            ret = ret[:-2]
    else:
        ret += "???"
    if return_result:
        return ret
    return ret



def has_variables(theorem):
    params = theorem[2]
    
    
    vars_theorem = filter(is_variable, params)
    
    for v in vars_theorem:
        if is_variable(v):
            return True
    return False


def same_constants_rules(conclusion, theorem):
    c_args = get_args(conclusion)
    t_args = get_args(theorem)
    
    for i in range(0, len(c_args)):
        if is_constant(c_args[i]) and is_constant(t_args[i]):
            if c_args[i] != t_args[i]:
                return False
            
    return True

# get(T, L) intoarce latura L din triunghiul T, unde latura 0 este între primele 2 varfuri, iar latura 1 intre al doilea si al treilea varf

def get(T, L):
    L = int(L)
    if L == 0:
        return T[0] + T[1]
    
    if L == 1:
        return T[1] + T[2]
    
    if L == 2:
        return T[0] + T[2]

# compute_triangle(L1, L2, L3) intoarce diferenta dintre suma laturilor mai scurte si latura cea mai lunga

def compute_triangle(L1, L2, L3):
    L1 = int(L1)
    L2 = int(L2)
    L3 = int(L3)    
    LS = getShortest(L1, L2, L3)
    LM = getMiddle(L1, L2, L3)
    LL = getLongest(L1, L2, L3)
    return LS + LM - LL

# getShortest intoarce cel mai mic dintre cele 3 argumente

def getShortest(LA, LB, LC):
    LA = int(LA)
    LB = int(LB)
    LC = int(LC)    
    sides = [LA, LB, LC]
    sides.sort()
    return sides[0]

# getMiddle intoarce argumentul median dintre cele 3 argumente

def getMiddle(LA, LB, LC):
    LA = int(LA)
    LB = int(LB)
    LC = int(LC)    
    sides = [LA, LB, LC]
    sides.sort()
    return sides[1]



# getLongest intoarce cel mai mare dintre cele 3 argumente

def getLongest(LA, LB, LC):
    LA = int(LA)
    LB = int(LB)
    LC = int(LC)
    sides = [LA, LB, LC]
    sides.sort()
    return sides[2]

# compute_pitagoras(LS, LM, LL) intoarce LS*LS + LM*LM - LL*LL

def compute_pitagoras(LS, LM, LL):
    LS = int(LS)
    LM = int(LM)
    LL = int(LL)
    return LS*LS + LM*LM - LL*LL


def get_function_result(f):
    f_name = get_head(f)
    f_args = get_args(f)
    
    if f_name == "get":
        return get(get_value(f_args[0]), get_value(f_args[1]))
    elif f_name == "compute_triangle":
        return compute_triangle(get_value(f_args[0]), get_value(f_args[1]), get_value(f_args[2]))        
    elif f_name == "getShortest":
        return getShortest(get_value(f_args[0]), get_value(f_args[1]), get_value(f_args[2]))        
    elif f_name == "getMiddle":
        return getMiddle(get_value(f_args[0]), get_value(f_args[1]), get_value(f_args[2]))        
    elif f_name == "getLongest":
        return getLongest(get_value(f_args[0]), get_value(f_args[1]), get_value(f_args[2]))        
    elif f_name == "compute_pitagoras":        
        return compute_pitagoras(get_value(f_args[0]), get_value(f_args[1]), get_value(f_args[2]))     

    
def fact_vars_the_same(fact):
    args = get_args(fact)
    
    if len(args) > 1:
        one_var = None
        for i in range (0, len(args)):
            if one_var == None:
                if is_variable(args[i]):
                    one_var = args[i]
            else:
                if is_variable(args[i]):
                    if get_name(args[i]) == get_name(one_var):
                        return get_name(one_var)
                    
    return False

    
    
def backward_chaining(kb, theorem, verbose):
    is_proved = False
    
    var_dict = {}
    unification_list = []
    CF = 1
    
    
    for fact in filter(is_fact, kb):
        if unify(fact, theorem) != False:
            CF = get_coef(fact)
            check_fact = fact_vars_the_same(fact)
            if check_fact == False:
                unification_list.append(unify(fact, theorem))
            else:
                subst_is = unify(fact, theorem)
                subst_is.pop(check_fact)
                unification_list.append(subst_is)
            print(print_formula(fact), end = '')
            print(" este un fapt" , " % ", CF)
            is_proved = True

            
    if is_proved:
        return (is_proved, unification_list, CF)
            
    check = False
    
    save_check = check
    temp_unification_list = []
    
    full_CF = 1
    full_CF_list = []
    
    for rule in filter(is_rule, kb):
        conclusion = get_conclusion(rule)
        save_check = check or save_check

        if (get_head(conclusion) == get_head(theorem) and len(get_args(conclusion)) == len(get_args(theorem))
            and same_constants_rules(conclusion, theorem)):

            print("Încercăm ", print_formula(rule), " % ", get_coef(rule))
            
            conclusion_CF = 1
            conclusion_CF_list = []
            dict_var_var = {}
            dict_var_val = {}
 
            conclusion_args = get_args(conclusion)
            theorem_args = get_args(theorem)
            
            
            for i in range(0, len(conclusion_args)):
                dict_var_var[conclusion_args[i]] = theorem_args[i]
            

            premises = get_premises(rule)

            check = True
            temp_unification_list = []
            
            for premise in premises:

                args_premiseBC = []
                args_premiseBC = deepcopy(get_args(premise))
                for j in range (0, len(args_premiseBC)):
                    if not is_function_call(args_premiseBC[j]):
                        if args_premiseBC[j] in dict_var_var.keys():

                            args_premiseBC[j] = dict_var_var[args_premiseBC[j]]
                    else:
                        f_args = get_args(args_premiseBC[j])

                        for k in range(0, len(f_args)):
                            if f_args[k] in dict_var_var.keys():                                
                                f_args[k] = dict_var_var[f_args[k]]
                            elif is_variable(f_args[k]):
                                if get_name(f_args[k]) in temp_unification_list[0]:
                                    f_args[k] = temp_unification_list[0][get_name(f_args[k])]
                        function_result = make_const(str(get_function_result(args_premiseBC[j])))
                        print(print_formula(args_premiseBC[j]), " => ", get_value(function_result))
                        args_premiseBC[j] = function_result

                premiseBC = (premise[0], premise[1], args_premiseBC)

                (verif, subst, trust_coef) = backward_chaining(kb, premiseBC, verbose)
                check = check and verif
                if check == False:
                    break
                conclusion_CF_list.append(trust_coef)

                if len(temp_unification_list) == 0:
                    if len(subst) > 0 and subst[0] != {}:
                        for i in subst:
                            temp_unification_list.append(i)

                else:
                    valid_substs = []
                    
                    for s in subst:
                        add_entry = False
                        var_not_added = False


                        for possibility in temp_unification_list:
                            var_not_added = False
                            for key in s:
                                if key not in possibility:  
                                    var_not_added = True
                                    break
                            if var_not_added:
                                can_add = True
                                for key in s:
                                    if key in possibility:
                                        if possibility[key] != s[key]: 
                                            can_add = False
                                if can_add:
                                    for key in s:
                                        if key not in possibility:
                                            possibility[key] = s[key]
                                    valid_substs.append(possibility)

                            else:
                                good_values = True
                                for key in s:
                                    if s[key] != possibility[key]:
                                        good_values = False
                                        break

                                if good_values:
                                    valid_substs.append(possibility)


                    temp_unification_list = valid_substs
           
                
            if check == True:

                for i in temp_unification_list:
                    unification_list.append(i)

                if len(conclusion_CF_list) > 0:
                    minim_premise_cf = conclusion_CF_list[0]
                    for premises_cf in conclusion_CF_list:
                        if minim_premise_cf > premises_cf:
                            minim_premise_cf = premises_cf
                            
                    
                    conclusion_CF = minim_premise_cf * get_coef(rule)
                    full_CF_list.append(conclusion_CF)
                    print(print_formula(conclusion), " % ", conclusion_CF)
                    

    if len(full_CF_list) == 1:
        full_CF = full_CF_list[0]
    elif len(full_CF_list) == 0:
        full_CF = 0
    else:
        product = 1
        sum_cf = 0
        for term in full_CF_list:
            product = product * term
            sum_cf = sum_cf + term
            
        full_CF = sum_cf - product
        
    
    save_check = check or save_check
    return (save_check, unification_list, full_CF)


def main():
    read_input("teste/testFN.txt")
    
    print("Fapte: ")
    for fact in facts:
        print(fact)
        
    print("Reguli: ")
    for rule in rules:
        print(rule)
        
    print("Intrebari: ")
    for question in questions:
        print(question)     
        
    print("------------")
    kb = get_kb(rules, facts)
        
        
    for question in questions:
        print("Scopuri de demonstrat: ", end = '')
        print(print_formula(question))

        local_kb = deepcopy(kb)
        (check, unif, CF) = backward_chaining(local_kb, question, True)
        print("Result")
        if not has_variables(question):
            print(check)
        else:
            print(unif)
            for u in unif:
                print(print_formula(substitute(question, u)))

        print("Trust coefficient : ", CF)
        print("------------------")
    

main()

Fapte: 
('ATOM', 'size', [('CONSTANT', 'AB'), ('CONSTANT', '3')], 1)
('ATOM', 'size', [('CONSTANT', 'BC'), ('CONSTANT', '4')], 1)
('ATOM', 'size', [('CONSTANT', 'AC'), ('CONSTANT', '5')], 1)
('ATOM', 'size', [('CONSTANT', 'AD'), ('CONSTANT', '4')], 1)
('ATOM', 'size', [('CONSTANT', 'CD'), ('CONSTANT', '3')], 1)
('ATOM', 'size', [('CONSTANT', 'BE'), ('CONSTANT', '3')], 1)
('ATOM', 'size', [('CONSTANT', 'CE'), ('CONSTANT', '3')], 1)
('ATOM', 'size', [('CONSTANT', 'AF'), ('CONSTANT', '2')], 1)
('ATOM', 'size', [('CONSTANT', 'BF'), ('CONSTANT', '1')], 1)
('ATOM', 'zero', [('CONSTANT', '0')], 1)
('ATOM', 'is', [('VARIABLE', '?X'), ('VARIABLE', '?X')], 1)
('ATOM', 'checks_out', [('CONSTANT', '1')], 1)
('ATOM', 'checks_out', [('CONSTANT', '2')], 1)
('ATOM', 'checks_out', [('CONSTANT', '3')], 1)
Reguli: 
('OR', 'OR', [('NOT', 'NOT', ('ATOM', 'size', [('FUNCTION', 'get', [('VARIABLE', '?T'), ('CONSTANT', '0')]), ('VARIABLE', '?LA')], 1)), ('NOT', 'NOT', ('ATOM', 'size', [('FUNCTION', 'get', [('